In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/Dataset_hyperEarlyStop

/content/drive/MyDrive/Dataset_hyperEarlyStop


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [ ]:
f=open("dataPartition.txt")
train=f.readline()
val=f.readline()
test=f.readline()
train=train[8:-2]
val=val[13:-2]
test=test[7:-2]
train=train.split(sep=', ')
for i in range(0,len(train),1):
  train[i]=int(train[i])
val=val.split(sep=', ')
for i in range(0, len(val),1):
  val[i]=int(val[i])
test=test.split(sep=', ')
for i in range(0, len(test),1):
  test[i]=int(test[i])

In [ ]:
t_l=pd.read_csv('train_loss.csv',skiprows=1)
e_l=pd.read_csv('eval_loss.csv')
e_a=pd.read_csv('eval_acc.csv')

In [ ]:
t_l_train=t_l[t_l.index.isin(train)].dropna()
t_l_val=t_l[t_l.index.isin(val)].dropna()
t_l_test=t_l[t_l.index.isin(test)].dropna()
e_l_train=e_l[e_l.index.isin(train)].dropna()
e_l_val=e_l[e_l.index.isin(val)].dropna()
e_l_test=e_l[e_l.index.isin(test)].dropna()
e_a_train=e_l[e_a.index.isin(train)].dropna()
e_a_val=e_a[e_a.index.isin(val)].dropna()
e_a_test=e_a[e_a.index.isin(test)].dropna()

In [ ]:
def create_dataset(dataset, window):
  X=[]
  Y=[]
  for i in range(0, len(dataset),1):
    for j in range(0, len(dataset[i])-window, 1):
      x=dataset[i,j:j+window]
      X.append(x)
      y=dataset[i,j+1:j+window+1]
      Y.append(y)
  return torch.tensor(X), torch.tensor(Y)

In [ ]:
class Model(nn.Module):
  def __init__(self, h_s):
    super().__init__()
    self.lstm=nn.LSTM(input_size=5, hidden_size=h_s, num_layers=1, batch_first=True)
    self.linear=nn.Linear(h_s, 1)
  def forward(self, x):
    x, _=self.lstm(x)
    x=self.linear(x)
    return x

In [ ]:
columns=list(e_a.columns)
columns=columns[4:]
t=e_a_train[columns].to_numpy()
X_train,Y_train=create_dataset(t,5)
v=e_a_val[columns].to_numpy()
X_val,Y_val=create_dataset(v,5)
test=e_a_test[columns].to_numpy()
X_test,Y_test=create_dataset(test,5)

<ipython-input-7-03d40300ff42>:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.tensor(X), torch.tensor(Y)


In [12]:
model1=Model(10)
model2=Model(30)
model3=Model(50)
model4=Model(70)
model5=Model(90)
models=[model1, model2, model3, model4, model5]
model_rmse=1000000
best_model=None
for model in models:
  optimizer=optim.Adam(model.parameters())
  loss_fn=nn.MSELoss()
  loader=data.DataLoader(data.TensorDataset(X_train,Y_train), shuffle=True)
  n_epochs=20
  for epoch in range(0, n_epochs, 1):
      model.train()
      for X_batch, Y_batch in loader:
          Y_pred = model(X_batch.to(torch.float32))
          loss = loss_fn(Y_pred, Y_batch.to(torch.float32))
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
      # Validation
      if epoch % 100 != 0:
          continue
      model.eval()
      with torch.no_grad():
          Y_pred = model(X_train.to(torch.float32))
          train_rmse = np.sqrt(loss_fn(Y_pred, Y_train))
          Y_pred = model(X_val.to(torch.float32))
          test_rmse = np.sqrt(loss_fn(Y_pred, Y_val))
      print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))
  model.eval()
  with torch.no_grad():
      Y_pred=model(X_val.to(torch.float32))
      test_rmse=np.sqrt(loss_fn(Y_pred, Y_val))
  if test_rmse<model_rmse:
      model_rmse=test_rmse
      best_model=model
torch.save(best_model,'model.h5')

Epoch 0: train RMSE 0.7908, test RMSE 0.5817
Epoch 0: train RMSE 0.9072, test RMSE 0.6375
Epoch 0: train RMSE 0.8625, test RMSE 0.5906
Epoch 0: train RMSE 0.7918, test RMSE 0.5085
Epoch 0: train RMSE 0.9027, test RMSE 0.5793


In [38]:
col1=columns[:5]
col2=columns[:10]
col3=columns[:20]
col4=columns[:30]
col5=columns[:60]
cols=[col1, col2, col3, col4, col5]
v=[5,10,20,30,60]
Y=torch.tensor(np.array(e_a_test['epoch_150']))
error=[]
for i in range(0,len(cols),1):
  col=cols[i]
  with torch.no_grad():
    test=torch.tensor(e_a_test.to_numpy())
    while(v[i]!=150):
      Y_pred=best_model(test[:,v[i]-5:v[i]].to(torch.float32))
      v[i]=v[i]+1
      if(v[i]==150):
        break
      test[:,v[i]]=Y_pred[:,0]
    rmse=np.sqrt(loss_fn(Y_pred,Y))
    error.append(rmse)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([49])) that is different to the input size (torch.Size([49, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [39]:
error

[tensor(3.8710, dtype=torch.float64),
 tensor(3.8635, dtype=torch.float64),
 tensor(3.8732, dtype=torch.float64),
 tensor(3.8756, dtype=torch.float64),
 tensor(3.8874, dtype=torch.float64)]